# Clone and install specified Python module

In [2]:
CHECK = "\u2705"
FAIL  = "\u274C"
WARN  = "\u26A0"
# --------------------------------------------------------------------
def git_clone(user, repo, folders):
    import os
    import subprocess

    homedir  = os.getcwd()
    repodir  = f'{homedir}/{repo}'
    
    print('BEGIN{clone and install}\n')
    url      = f'https://github.com/{user}/{repo}'
    raw_url  = url.replace('github.com', 'raw.githubusercontent.com')
    raw_url += '/refs/heads/main/'
    toml_file= 'pyproject.toml'

    # 1. Delete local repo folder if it exists
    print(f'\t1. delete old repo: {repo}')
    result = subprocess.run(
        ["rm", "-rf", repo],
        capture_output=True)
    if result.returncode != 0:
        print(f"{FAIL} Deletion of {repo} failed:",
              result.stderr.strip())
        return result.returncode

    # 2. Clone repo <repo> with minimal history
    print(f'\t2. clone repo: {repo}')
    result = subprocess.run(
        ["git", "clone",
         "--depth", "1",
         "--filter=blob:none",
         "--sparse", url],
        capture_output=True,
        text=True)
    if result.returncode == 0:
        print(result.stdout.strip())
    else:
        print(f"{FAIL} Clone of {repo} failed:", result.stderr.strip())
        return result.returncode

    # 3. Checkout specified folders
    print(f'\t3. cd to repo folder: {repo}')
    os.chdir(repodir) # move to repo folder
    for folder in folders:
        print(f'\t4. check out folder: {folder}')
        result = subprocess.run(
        	["git", "sparse-checkout",
          	"set", folder],
        	capture_output=True,
        	text=True)
        if result.returncode == 0:
        	print(result.stdout.strip())
        else:
            print(f"{FAIL} Sparse checkout of {dirname} failed:",
                  result.stderr.strip())
            os.chdir(homedir)
            return result.returncode

    # 5. wget toml_file
    os.chdir(repodir) # make sure we're in repo folder
    print(f'\t5. wget {toml_file}')
    url = raw_url + toml_file
    result = subprocess.run(
        ["wget", "-q", url, "-O", toml_file],
        capture_output=True, text=True)
    if result.returncode != 0:
        print(f"{FAIL} wget of {toml_file} failed:", result.stderr.strip())
        return result.returncode

    # 6. pip install -e . module
    print(f'\t6. pip install -e .\n')
    os.chdir(repodir) # make sure we're in repo folder
    !pip uninstall -y pinn4bhoc
    !pip install -e .

    import importlib
    try:
        mod = importlib.import_module(repo)
        print(f"✅ Successfully imported {repo}")
    except ModuleNotFoundError as e:
        print(f"❌ Failed to import '{repo}': {e}")

    # result = subprocess.run(
    #     ["pip", "install", "-e", '.'],
    #     capture_output=True,
    #     text=True)
    # if result.returncode == 0:
    #     print(result.stdout.strip())
    # else:
    #     print(f"{FAIL} pip install -e . failed: ",
    #           result.stderr.strip())
    #     os.chdir(homedir)
    #     return result.returncode
        
    print('\nEND{clone and install}')
    

    os.chdir(homedir)
    
    return 0

In [3]:
if IN_COLAB:
    git_clone(GITHUB_USER, GITHUB_REPO, GITHUB_FOLDERS)

Running locally
